In [ ]:
# figure 6A: measuring likelihood of shared and ind gsm over bsd500 natural images
# author: Amir Farzmahdi
# last update: June 17th, 2024

In [ ]:
# library imports
import os
import random
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pyrtools as pt
import time
import scipy as sp
import pandas as pd
import math

In [ ]:
# set random seed
np.random.seed(42)
random.seed(42)

In [ ]:
# setting

# filters parameters
n_loc = 17 
n_theta = 15
g_num_filts = 36
ncs_s = 36
ncs_i = 36
ntest = 10000 

In [ ]:
# load covariance matrix
with open('cov_mat_test_res.csv', "rb") as fp:
    model_res = pickle.load(fp)  
train_cov_mat = model_res['train_cov_mat']
test_filter_sz_res = model_res['test_filter_sz_res']

In [ ]:
# geometry mean function
def geo_mean_overflow(iterable):
    return np.exp(np.log(iterable).mean())

In [ ]:
# generate a mask for cov matrix of independent case
mask = np.zeros((g_num_filts,g_num_filts))
mask[:int(g_num_filts/2),:int(g_num_filts/2)] = 1
mask[int(g_num_filts/2):,int(g_num_filts/2):] = 1

In [ ]:
# natural image likelihood, based on Coen-Cagli et. al (2009)
p_indp = []
p_shared = []

p_shared_locs_imgs = np.zeros((n_loc,n_theta))
p_indp_locs_imgs = np.zeros((n_loc,n_theta))
p_diff = np.zeros((n_loc,n_theta))
for i_loc in range(0,n_loc):
    for i_theta in range(0,n_theta):
        idx = np.linspace((i_theta*g_num_filts),(i_theta+1)*g_num_filts-1,g_num_filts).astype(int)
        p_indp = []
        p_shared = []
        
        # shared
        cov_mat_shared = train_cov_mat[i_loc,i_theta,:,:]
        inv_cov_mat_shared = np.linalg.inv(cov_mat_shared)
        det_inv_cov_mat_shared = np.linalg.det(inv_cov_mat_shared)
        
        # independent
        cov_mat_indp = cov_mat_shared * mask
        inv_cov_mat_indp = np.linalg.inv(cov_mat_indp)
        det_inv_cov_mat_indp = np.linalg.det(inv_cov_mat_indp)
        
        # loop over entire test images
        for i in range(0, ntest):
            img_res = test_filter_sz_res[i,i_loc,idx]
            
            # shared
            lambda_ = np.sqrt(img_res.T @ inv_cov_mat_shared @ img_res) 
            p_shared.append((np.power(det_inv_cov_mat_shared,1/2)/np.power(2*math.pi,ncs_s/2)) 
                            * (sp.special.kv((1-ncs_s/2),lambda_)/np.power(lambda_,ncs_s/2-1)))
            # independent
            lambda_ = np.sqrt(img_res.T @ inv_cov_mat_indp @ img_res)
            p_indp.append((np.power(det_inv_cov_mat_indp,1/2)/np.power(2*math.pi,ncs_i/2)) 
                          * (sp.special.kv((1-ncs_i/2),lambda_)/np.power(lambda_,ncs_i/2-1)))

        p_shared_locs_imgs[i_loc,i_theta] = geo_mean_overflow(p_shared)
        p_indp_locs_imgs[i_loc,i_theta] = geo_mean_overflow(p_indp)
        p_diff[i_loc,i_theta] = np.log(geo_mean_overflow(p_shared)) - np.log(geo_mean_overflow(p_indp))

In [ ]:
# save p_diff
with open(f'p_diff_nat_images_{n_loc}_locs_{n_theta}_oris.csv', "wb") as fp:  
    pickle.dump(dict(p_diff=p_diff),fp)